In [ ]:
!python3 ~/Documents/thesis/GroupEquivariantChebNets/scripts/debug_script.py

In [ ]:
import torch
from torch.nn import BatchNorm1d, Module, MaxPool1d, AvgPool1d

from gechebnet.graph.graph import SE2GEGraph
from gechebnet.model.chebnet import GEChebNet

In [ ]:
import torch
from pykeops.torch import LazyTensor, Pm, Vi, Vj
import math

In [ ]:
def so3_matrix(alpha, beta, gamma, device=None):
    cosa = torch.cos(alpha)
    sina = torch.sin(alpha)
    Ra = torch.zeros((alpha.shape[0], 3, 3), device=device)
    Ra[:, 1, 1] = cosa
    Ra[:, 1, 2] = -sina
    Ra[:, 2, 1] = sina
    Ra[:, 2, 2] = cosa
    Ra[:, 0, 0] = 1.0

    cosb = torch.cos(beta)
    sinb = torch.sin(beta)
    Rb = torch.zeros((beta.shape[0], 3, 3), device=device)
    Rb[:, 0, 0] = cosb
    Rb[:, 0, 2] = sinb
    Rb[:, 2, 0] = -sinb
    Rb[:, 2, 2] = cosb
    Rb[:, 1, 1] = 1.0

    cosc = torch.cos(gamma)
    sinc = torch.sin(gamma)
    Rc = torch.zeros((gamma.shape[0], 3, 3), device=device)
    Rc[:, 0, 0] = cosc
    Rc[:, 0, 1] = -sinc
    Rc[:, 1, 0] = sinc
    Rc[:, 1, 1] = cosc
    Rc[:, 2, 2] = 1.0

    return Rc @ Rb @ Ra



In [ ]:
0.3232**2 + 0.3232**2

In [ ]:
alpha = torch.tensor([math.pi/4])
beta = torch.tensor([math.pi/4])
gamma = torch.tensor([math.pi/4])

In [ ]:
Gg = so3_matrix(alpha, beta, gamma)[0]
Gg

In [ ]:
Gg = torch.tensor([
    [-0.499861, -0.0920301, -0.861202],
    [-0.263421, -0.931079, 0.252393], 
    [-0.825075, 0.35302, 0.441167]
])
Gg

In [ ]:
A = 0.5 * (Gg - Gg.t())
A

In [ ]:
norm_A = (0.5 * A.pow(2).sum()).sqrt()
norm_A

In [ ]:
Log = A * torch.asin(norm_A) / norm_A
math.acos(1)

In [ ]:
theta = math.acos((Gg.diag().sum() - 1)/2)
Log = theta / math.sin(theta) * 0.5 * (Gg - Gg.t())
Log

In [ ]:
0.406123 / math.asin(0.406123)

In [ ]:
import torch
import math

In [ ]:
x = torch.rand(10)
y = torch.rand(10)
z = torch.rand(10)

norm = torch.sqrt(x.pow(2) + y.pow(2) + z.pow(2))

x *= math.pi / norm
y *= math.pi / norm
z *= math.pi / norm

In [ ]:
x = torch.tensor([1])
y = torch.tensor([0])

torch.atan2(x, y)

In [ ]:
def xyz2alphabetagamma(x, y, z):
    # atan2 always returns a number in [-pi ; pi]
    
    alpha = torch.sqrt(x.pow(2) + y.pow(2) + z.pow(2)) - math.pi #ok
    
    beta = torch.stack(
        (torch.atan2(-z, -torch.sqrt(x.pow(2) + y.pow(2))), torch.atan2(-z, torch.sqrt(x.pow(2) + y.pow(2)))),
        dim=-1
    )
    
    gamma = torch.stack(
        (torch.atan2(-y, -x), torch.atan2(y, x)),
        dim=-1
    )
    
    mask = (beta >= -math.pi) & (beta < math.pi) & (gamma >= -math.pi/2) & (gamma < math.pi/2)
        
    return alpha, beta[mask], gamma[mask]
    
def alphabetagamma2xyz(alpha, beta, gamma):
    x = (math.pi + alpha) * torch.cos(beta) * torch.cos(gamma)
    y = (math.pi + alpha) * torch.cos(beta) * torch.sin(gamma)
    z = -(math.pi + alpha) * torch.sin(beta)
    return x, y, z
    
def mod(input, n, d=0.0):
    """
    Returns a new tensor with the modulo with offset of the elements of input.

    Args:
        input (Tensor): input tensor.
        n (float): modulus.
        d (float, optional): offset. Defaults to 0.0.

    Returns:
        (Tensor): output tensor.
    """
    return (input - d) % n + d

In [ ]:
a = torch.tensor([-1., 0., 1.])
x, y, z = torch.meshgrid(a,a,a)

In [ ]:
x, y, z = x.flatten(), y.flatten(), z.flatten()

mask = (x == 0.) & (y == 0.) & (z == 0.)
x, y, z = x[~mask], y[~mask], z[~mask]

norm = torch.sqrt(x.pow(2) + y.pow(2) + z.pow(2))

x *= math.pi / norm
y *= math.pi / norm
z *= math.pi / norm

x, y, z

In [ ]:
alpha, beta, gamma = xyz2alphabetagamma(x, y, z)
alpha, beta, gamma

In [ ]:
x_, y_, z_ = alphabetagamma2xyz(alpha, beta, gamma)

In [ ]:
torch.allclose(x, x_, atol=1e-5, equal_nan=True), torch.allclose(y, y_, atol=1e-5, equal_nan=True), torch.allclose(z, z_, atol=1e-5, equal_nan=True)

In [ ]:
torch.allclose(x, x_, atol=1e-5, equal_nan=True), torch.allclose(y, y_, atol=1e-5, equal_nan=True), torch.allclose(z, z_, atol=1e-5, equal_nan=True)

In [ ]:
alpha, beta, gamma = xyz2alphabetagamma(x, y, z)
alpha, beta, gamma

In [ ]:
x, y, z = alphabetagamma2xyz(alpha, beta, gamma)
x, y, z

In [ ]:
xi = 0.6545
eps = .58

graph = SE2GEGraph(
    grid_size=(28, 28),
    nsym=9,
    knn=4,
    sigmas=(xi / eps, xi, 1.0),
    weight_kernel=lambda sqdistc, sigmac: torch.exp(-sqdistc / sigmac),           # gaussian
    #weight_kernel = lambda sqdistc, sigmac: torch.exp(-torch.sqrt(sqdistc/sigmac)), # laplace
    #weight_kernel = lambda sqdistc, sigmac: 1 / (1 + sqdistc / sigmac),        # cauchy
    kappa=0.6657,
)

graph.num_edges

In [ ]:
model = GEChebNet(graph, K=10, in_channels=1, out_channels=10, hidden_channels=20, pooling="avg", laplacian_device=torch.device("cuda"))
model = model.to(torch.device("cuda"))

In [ ]:
x = torch.rand(4, 1, 28*28*9, device=torch.device("cuda"))

In [ ]:
model(x)

In [ ]:
D = torch.sparse.FloatTensor(edge_index, edge_weight, torch.Size((num_nodes, num_nodes)))

In [ ]:
D @ torch.ones(num_nodes)

In [ ]:
torch.zeros(num_nodes).scatter_add(0, edge_index[1], edge_weight)

In [ ]:
edge_index_t = edge_index.t()
max_, _ = torch.max(edge_index_t, dim=1)
min_, _ = torch.min(edge_index_t, dim=1)
minmax = torch.stack((min_, max_), dim=0)

_, inverse, counts = torch.unique(minmax, dim=1, return_inverse=True, return_counts=True)
bad_index = torch.arange(counts.shape[0])[(counts < 2)]

In [ ]:
(counts < 2).sum()

In [ ]:
edge_index = torch.tensor([[0, 1, 2, 3, 1], [1, 0, 3, 2, 1]])
edge_weight = torch.tensor([0.5, 0.5,0.5,0.5, 0.4])

In [ ]:
edge_index_t = edge_index.t()
max_, _ = torch.max(edge_index_t, dim=1)
min_, _ = torch.min(edge_index_t, dim=1)
minmax = torch.stack((min_, max_), dim=0)

_, inverse, counts = torch.unique(minmax, dim=1, return_inverse=True, return_counts=True)
bad_index = torch.arange(counts.shape[0])[(counts < 2)]

In [ ]:
(counts < 2).sum()

In [ ]:
minmax

In [ ]:
from torch import LongTensor, FloatTensor
from typing import Tuple

def remove_self_loops(edge_index: LongTensor, edge_weight: FloatTensor) -> Tuple[LongTensor, FloatTensor]:
    """
    Removes every self-loop in the graph given by edge_index and edge_weight.

    Args:
        edge_index (LongTensor): indices of edges.
        edge_weight (FloatTensor): weights of edges.

    Returns:
        LongTensor: indices of edges.
        FloatTensor: weights of edges.
    """
    mask = edge_index[0] != edge_index[1]
    return edge_index[:, mask], edge_weight[mask]


def is_undirected(edge_index: LongTensor, edge_weight: FloatTensor) -> bool:
    """
    Returns True if the graph given by edge_index and edge_weight is undirected.

    Args:
        edge_index (LongTensor): indices of edges.
        edge_weight (FloatTensor): weights of indices.
        num_nodes (int): number of nodes.

    Returns:
        (bool): True if graph is undirected, False otherwise.
    """
    
    if (edge_index[0] == edge_index[1]).sum() > 0:
        print("Warning: Graph contains self-loop")    
        edge_index, edge_weight = remove_self_loops(edge_index, edge_weight)        
        
    max_, _ = torch.max(edge_index, dim=0)
    min_, _ = torch.min(edge_index, dim=0)
    minmax = torch.stack((min_, max_), dim=0)
    
    _, inverse, counts = torch.unique(minmax, dim=1, return_inverse=True, return_counts=True)

    return (counts < 2).sum() == 0

In [ ]:
is_undirected(edge_index, edge_weight)

In [ ]:
edge_index = torch.tensor([[0, 0, 2, 3, 0], [2, 3, 3, 2, 3]])
edge_weight = torch.tensor([0.5, 0.5, 0.5, 0.4])

In [ ]:
edge_index.t()

In [ ]:
sorted, indices = edge_index.t().sort(dim=1)

In [ ]:
sorted

In [ ]:
indices

In [ ]:
import pandas as pd

In [ ]:
array = edge_index.numpy()

df1 = pd.DataFrame(np.arange(5), array[0], array[1])
df1

In [ ]:
array[0]